Importing the libraries that will be used in the notebook

In [1]:
# Tools used to WebSrapping
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
# Tools used to data analysis
import time
import pandas as pd
import json
import math
import itertools
import random

### 1. This notebook section is responsible for collecting the URLS and the number of employees of each of the selected companies.

In [2]:
companies_list=list(pd.read_excel("companiy_list.xlsx")["companies"])

1.1 Using a bot to login into Linkedin

In [ ]:
driver  = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(1920, 1800)
driver.maximize_window()
email= "email" 
password = "password"

driver.get("https://www.linkedin.com/login")
time.sleep(2)

eml = driver.find_element(by=By.ID, value="username")
eml.send_keys(email)
passwd = driver.find_element(by=By.ID, value="password")
passwd.send_keys(password)
loginbutton = driver.find_element(by=By.XPATH, value="//*[@id=\"organic-div\"]/form/div[3]/button")
loginbutton.click()
time.sleep(3)

1.2 Searching for the companies profiles.

In [4]:
#This cell finds the url's of most companies, but some are not found, because Linkedin complicates the collection of information from profiles, so the next cell works to find the ones that were not found in this first one.
companies_dict = {}
not_found_list1=[]
def finding_jobs():
        for i in companies_list:
            try:
                driver.get("https://www.linkedin.com/search/results/companies/?keywords={}&sid=LeK".format(i))
                time.sleep(3)
                companies_dict[i]=driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[1]/ul/li[1]/div/div/div[2]/div[1]/div[1]/div/span/span/a' ).get_attribute("href")
                time.sleep(3)
            except:
                not_found_list1.append(i)
                pass
finding_jobs()

In [5]:
#This cell ends the job of finding the url's of the companies. It's important to notice that not all companies will be found. That can happen becaue the companie name is not the same as the name in Linkedin, or because the company is not in Linkedin.
wrong_names_list=[]
def finding_jobs2():
    for i in not_found_list1:
        try:
            driver.get("https://www.linkedin.com/search/results/companies/?keywords={}&sid=LeK".format(i))
            time.sleep(2)
            companies_dict[i]=driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div[2]/div/div[1]/main/div/div/div[1]/ul/li[1]/div/div/div[2]/div[1]/div[1]/div/span/span/a' ).get_attribute("href")
            time.sleep(2)
        except:
            wrong_names_list.append(i)
            pass
finding_jobs2() 

In [6]:
#This cell will convert the dictionary into a dataframe, to facilitate the analysis of the data.
companies_url=pd.DataFrame(companies_dict.items(), columns=["companies", "url"])

In [9]:
# Once we got the URLS of the companies, we can start to collect the data from their profiles. This cell is used to collect the numbers of employees of the company.
companies_url_dict={}
companies_name_dict={}
dict_not_found={}
list_erro=[]
def getting_profiles():
    for i in range(len(companies_url)):
        driver.get(companies_url["url"][i]+"people/")
        time.sleep( random.uniform(2, 3))
        driver.execute_script("window.scrollTo(0, 100);")
        try:
            source = BeautifulSoup(driver.page_source, "html.parser")
            trab=""
            info = source.find('div', class_="display-flex full-width justify-space-between align-items-center pt5 ph5")
            for m in info.find("h2",class_="t-20 t-black t-bold").get_text().strip():
                if m.isdigit():
                    trab+=m
            companies_url_dict[companies_url["url"][i]]=companies_name_dict[companies_url["companies"][i]]=trab
        except:
            list_erro.append(companies_url["companies"][i])
            pass
getting_profiles()
                

In [11]:
#Creating the final DataFrame with the companie name, the url and the number of employees in the company.
df=companies_url.merge(pd.DataFrame(companies_url_dict.items(), columns=["url", "number_of_employees"]))

In [ ]:
#Converting the number of employees to a numeric value. And sorting values
def convert (x):
    if x.isdigit():
        return int(x)
    else:
        return 0
df["number_of_employees"]=df["number_of_employees"].apply(convert)
df=df.sort_values(by="number_of_employees", ascending=False)

### 2. This notebook section will collect the URL for the employees of each companie

2.1 The next cell will gather all the URLS from each employee of each company

In [25]:
df=df.tail(40)[:3]

In [26]:
urls=[]
companie=[]
for i,m, w in zip(df["url"],df["number_of_employees"],df["companies"]):
    driver.get( i +"/people")
    time.sleep(random.uniform(2, 3))
    # Linkedin doesn't automatically show all profiles, so we need to scroll up and down the page to make the data appear. 
    # For each company, we need to scroll up and down the page the number of times that is equal to the number of employees divided by 12, because Linkedin shows 12 profiles per page.
    # The number of employees divided by 12 is rounded up, because we need to make sure that we will get all the profiles.
    if int(m)<1020:
        c=0
        while c<int(m)/12+1:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(0.5, 2))
            driver.execute_script("window.scrollTo(0, -1);")
            time.sleep(random.uniform(0.5, 2))
            c+=1
    else:
        c=0
        # Linkedin shows a maximum of something like 1000 profiles, so we need to make sure that we will not scroll a lot more than 1000 times. Seems that 1030 is the maximum number of scorlls that we may need to do.
        while c<1030/12+1:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(0.5, 2))
            driver.execute_script("window.scrollTo(0, -1);")
            time.sleep(random.uniform(0.5, 2))
            c+=1
    #Finally, we get the url of each profile and add it to a list. Is important to get company name and url, because we will use it later to create a DataFrame.
    for j in range(1,int(m)+1):
        try:
            urls.append(driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div[2]/div/div[2]/main/div[2]/div/div[2]/div/div[1]/ul/li[{}]/div/section/div/div/div[2]/div[1]/a".format(str(j))).get_attribute("href"))
            companie.append(w)
        except:
            pass

### 3. In this final section we will gather data from each profile

3.1 The three following fucntions ( **Type2Jobs**, **ParseType1Job**, **returnProfileInfo** ) were found at: https://github.com/thelazyaz/linkedin-web-scraping, under no specific license. I add somechanges to fit my specific problem, but most of the code is the same. 

In [27]:
# Function to get information about the profiles. It will get the data from jobs that were done in group. The information that will be gathered is: company, duration and the jobgroup.
def Type2Jobs(alltext):
    jobgroups = []
    company = alltext[16][:len(alltext[16]) // 2]
    totalDurationAtCompany = alltext[20][:len(alltext[20]) // 2]

    groups = []
    count = 0
    index = 0
    for a in alltext:
        if a == '' or a == ' ':
            count += 1
        else:
            groups.append((count, index))
            count = 0
        index += 1

    numJobsInJoblist = [g for g in groups if g[0] == 21 or g[0] == 22 or g[0] == 25 or g[0] == 26]
    for i in numJobsInJoblist:
        if 'time' in alltext[i[1] + 5][:len(alltext[i[1] + 5]) // 2].lower().split('-'):
            jobgroups.append((alltext[i[1]][:len(alltext[i[1]]) // 2], alltext[i[1] + 8][:len(alltext[i[1] + 8]) // 2]))
        else:
            jobgroups.append((alltext[i[1]][:len(alltext[i[1]]) // 2], alltext[i[1] + 4][:len(alltext[i[1] + 4]) // 2]))
    return ('type2job', company, totalDurationAtCompany, jobgroups)

# Function to get information about the profiles. It will get the data from individual jobs.  The information that will be gathered is the Job Title, Company, Duration, Location
def parseType1Job(alltext):
    jobtitle = alltext[16][:len(alltext[16]) // 2]
    company = alltext[20][:len(alltext[20]) // 2]
    duration = alltext[23][:len(alltext[23]) // 2]
    location = alltext[26][:len(alltext[26]) // 2]
    return ('type1job', jobtitle, company, duration, location)

#Function to get the information from the profiles. It concate data that were gathered from the last two functions. 
# Additionally, it will get the name of the profile, experience, education, skills and certifications of each profile.
def returnProfileInfo(employeeLink, companyName):
    try:
        url = employeeLink
        driver.get(url)
        time.sleep(random.uniform(2, 3))
        source = BeautifulSoup(driver.page_source, "html.parser")

        profile = []
        profile.append(companyName)
        info = source.find('div', class_='mt2 relative')
        name = info.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').get_text().strip()
        title = info.find('div', class_='text-body-medium break-words').get_text().lstrip().strip()
        location = info.find('span', class_='text-body-small inline t-black--light break-words').get_text().strip()
        profile.append(name)
        profile.append(title)
        profile.append(location)
        time.sleep(random.uniform(1, 2))
        experiences = source.find_all('li', class_='artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column')

        #Getting information about the experiences
        for x in experiences[1:]:
            alltext = x.getText().split('\n')
            print(alltext)
            startIdentifier = 0
            for e in alltext:
                if e == '' or e == ' ':
                    startIdentifier+=1
                else:
                    break
            if startIdentifier == 16:
                # Getting information about the educaticon
                if 'universidade' in alltext[16].lower().split(' ') or 'faculdade' in alltext[16].lower().split(' ') or 'ba' in alltext[16].lower().split(' ') or 'bs' in alltext[16].lower().split(' '):
                    profile.append(('education', alltext[16][:len(alltext[16])//2], alltext[20][:len(alltext[20])//2]))

                # CGetting information about the certifications
                elif 'Competências' in alltext[23].lower().split(' '):
                    profile.append(('certification', alltext[16][:len(alltext[16])//2], alltext[20][:len(alltext[20])//2]))

            elif startIdentifier == 12:
                # Getting information about the skills
                if (alltext[16] == '' or alltext[16] == ' ') and len(alltext) > 24:
                    profile.append(('skill', alltext[12][:len(alltext[12])//2]))

        # Getting information about the Jobs
        url = driver.current_url + '/details/experience/'
        driver.get(url)
        time.sleep(random.uniform(2, 3))
        source = BeautifulSoup(driver.page_source, "html.parser")
        time.sleep(random.uniform(1, 2))
        exp = source.find_all('li')
        for e in exp[13:]:
            row = e.getText().split('\n')
            if row[:16] == ['', '', '', '', '', '', ' ', '', '', '', '', '', '', '', '', '']:
                if 'yrs' in row[20].split(' '):
                    profile.append(parseType2Jobs(row))
                else:
                    profile.append(parseType1Job(row))
        return profile
    except:
        pass

3.2 Using the functions to gather data about each of the profiles that we collected.

In [ ]:
prof=[]
for url, emp in zip(urls, companie):
    prof.append(returnProfileInfo(url, emp))

3.3 Converting each profile to a row of a DataFrame

In [31]:
info=[]
for i in range(len(prof)):
    try:
        skill=[]
        job=[]
        char=[]
        for i in prof[i]:
            if type(i)==tuple:
                if "skill" in i:
                    skill.append(i[1])
                elif "type1job" in i:
                    job.append(i[1:])
            else:
                char.append(i)
        # We will limit the number of jobs to the first 5 that each profile has.
        if len(job)<5: 
            while len(job)!=5:
                job.append("")
        else:
            job=job[:5]

        # We will limit the number of skills to the first 5 that each profile has.
        if len (skill)<5:
            while len(skill)!=5:
                skill.append("")
        else:
            skill=skill[:5]
        info.append(char+skill+job)
    except:
        pass
    colunas=["Companie","Name","Job Title","Location","Skill1","Skill2","Skill3","Skill4","Skill5","Current Job","Past Job1","Past Job2","Past Job3","Past Job4"]
#  Converting the data into a dataframe
base= pd.DataFrame(info,columns=colunas)